[Go to Home Page](/weaviate/lab/workspaces/auto-3/tree/main.ipynb)


# Bedrock Model Invocation
This notebook contains a Python script to invoke various Bedrock models. The script defines functions to select and interact with different AI models like COHERE, TITAN, CLAUDE, and others, using the Bedrock API.

# Listing Bedrock Models
Your SageMaker notebook comes with full access to all Amazon Bedrock Models. In addition, an IAM role has been associated with the notebook to allow seamless invocation without having to pass AWS credentials explicitly.

Follow the steps below to list all available Bedrock models.

In [ ]:
import boto3
import pandas as pd

def list_bedrock_models():
    try:
        client = boto3.client('bedrock')
        response = client.list_foundation_models()
        
        # Prepare data for pretty display
        model_data = []
        for model in response['modelSummaries']:
            model_data.append({
                'Model Name': model['modelName'],
                'Provider Name': model['providerName']
            })
        
        # Convert to DataFrame for pretty display
        df = pd.DataFrame(model_data)
        return df
        
    except Exception as e:
        print(f"Error listing models: {str(e)}")

# Display the table
df = list_bedrock_models()
if df is not None:
    from IPython.display import display
    pd.set_option('display.max_rows', None)  # Set to display all rows
    display(df)


# Invoking Bedrock Models
Next, you can invoke different Bedrock models using the provided function. The function `invoke_bedrock_model` allows you to specify the model type and various parameters for invocation.

In [ ]:
import boto3
import json

def invoke_bedrock_model(model_type, template, max_tokens, temperature, top_p, stop_sequences):
    """
    Invoke a Bedrock model based on the given model type and parameters.

    Parameters:
    - model_type (str): The type of the model (e.g., "TITAN", "CLAUDE", etc.)
    - template (str): The prompt for the model.
    - max_tokens (int): Maximum number of tokens to sample.
    - temperature (float): Sampling temperature.
    - top_p (float): Top-p sampling value.
    - stop_sequences (list): List of sequences to stop the sampling.

    Returns:
    - None: Prints the model's completion.
    """

    # Determine the model ID based on the provided model type
    if model_type == "TITAN":
        model_id = "amazon.titan-tg1-large"
    elif model_type == "CLAUDE":
        model_id = "anthropic.claude-v1"
    elif model_type == "CLAUDE-INSTANT":
        model_id = "anthropic.claude-instant-v1"
    elif model_type == "J2":
        model_id = "ai21.j2-jumbo-instruct"
    else:
        # Handle unsupported models
        raise Exception("Unsupported model")

    # Initialize the Bedrock client
    bedrock = boto3.client('bedrock-runtime')

    # Prepare the request body with the parameters
    body = {
        "prompt": template,
        "max_tokens_to_sample": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "stop_sequences": stop_sequences
    }
    body_string = json.dumps(body)

    # Invoke the Bedrock model and get the response
    response = bedrock.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=body_string)

    # Parse the response and print the completion
    json_obj = json.loads(response.get("body").read().decode())
    print(json_obj['completion'])


def bedrock_smoke_test():
    # Example usage of the invoke_bedrock_model function
    model_type="CLAUDE-INSTANT"
    prompt = "What does Weaviate do? What are the top use cases? How can Amazon Bedrock be used with Weaviate?"
    template = f"""Human: {prompt}
                    Assistant:"""
    max_tokens = 5000
    stop_sequences=[]
    temperature=0
    top_p=0.9
    invoke_bedrock_model(model_type, template, max_tokens, temperature, top_p, stop_sequences)

# Run the smoke test when the script is executed
bedrock_smoke_test()


[Go to Home Page](/weaviate/lab/workspaces/auto-3/tree/main.ipynb)
